In [1]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt 


dataset = ['EFIFI','Nair'][1]
basedir = ['../../bf_data/EFIGI_catalog/','../../bf_data/Nair_and_Abraham_2010/'][1]
fitsdir = basedir + ['fits_temp_Jan_19/','fits_temp_Dec_28/', 'fits_temp_Feb_3/', 'final/'][3]

out_dir = basedir+'out_final/'
fns_r = glob(fitsdir+"*/*r.fits")
fns_r.sort()

# Generate mask + Initial StatMorph

In [2]:
from astrobf.utils import gen_mask
from astrobf import measure_morph

In [ ]:
gen_mask.run_mask(fns_r, out_dir, npix_min=40)

/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log1

0-th chunk done


/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log1

1-th chunk done


/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log1

...

/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log1

14-th chunk done


/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log1

15-th chunk done


/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))


In [ ]:
fns_r[]

In [6]:
len(fns_r)

14034

In [14]:
fns_sub = fns_r[767*9:]

In [15]:
from multiprocessing import Pool

def run(fns_r):
    return gen_mask.run_mask(fns_r, out_dir, npix_min=40)

if __name__ == '__main__':
    with Pool(2) as p:
        print(p.map(run, [fns_sub[:3510],fns_sub[3510:]]))

/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log1

0-th chunk done


/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))


0-th chunk done


/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log1

...

400-th chunk done


/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log1

401-th chunk done


/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))


402-th chunk done
[None, None]


So far, So good.

In [ ]:
# Run StatMorph for the first time
# Just in case, only for this first run, I pickle all 'morph' instances. a few 10s of GB in total.
measure_morph.run_stat_morph_init(fns[:10], out_dir, eps=1e-6)

# Visual inspection - Filter out bad samples
Keep good samples under `.../stat_png/good/*.png`

In [ ]:
all_morphs = glob(out_dir+"Morphs/final_morphs*.pickle")
all_morphs.sort()
for alm in all_morphs:
    print(alm)
    mps = pickle.load(open(alm,'rb'))
    for morph in mps:
        img_name = morph._gid
        fn = get_fn(img_name)
            
        hdulist = fits.open(fn)
        img = hdulist[0].data
        img -= (img.min() - eps)
        
        image_diagnostics.make_figure(morph, nrows=3,
                                      savefig=gen_fn_img(morph), img_org=None)

In [165]:
good_list = glob(basedir+'out1/'+"stat_png/good/*.png")
good_gids=[]
for gg in good_list:
    good_gids.append(gg.split("/")[-1].split("_")[1])
good_gids.sort()
good_fns = [glob(fitsdir+gg+'/*r.fits')[0] for gg in good_gids]

#### Following part will be updated if I have to go through this step again.

In [ ]:
# Catalog of StatMorph properties of the good sample.
from astrobf.utils.measure_morph import statMorph_fields
fields = statMorph_fields[:6]
ngal = len(good_gids)
result_arr = np.zeros(ngal, 
                      dtype=[('id','<U24'),('ttype',int),('flag',bool),('flag_sersic',bool)]
                           +[(ff,float) for ff in fields])
i=0
for alm in all_morphs:
    mps = pickle.load(open(alm,'rb'))
    for morph in mps:
        if morph._gid in good_gids:
            result_arr[i]['id'] = morph._gid
            for ff in fields:
                result_arr[i][ff] = getattr(morph, ff)
            i+=1
        else:
            pass

# Merge files to minimize I/O

In [57]:
from astropy.io import fits
import pickle
from astrobf.utils import mask_utils

all_data=[]
for fn in good_fns:
    img_name = fn.split("/")[-2]
    img = fits.open(fn_img)[0].data
    weight = fits.open(fn_weight)[0].data
    mask = pickle.load(open(out_dir+f"/masks/{img_name}_mask.pickle", 'rb'))
    slices = mask_utils._get_stamp_range(mask)
    data = np.stack((img[slices], mask[slices], weight[slices]))
    all_data.append({'data':data, 'img_name':img_name, 'slices':slices})

pickle.dump(all_data, open("all_gals.pickle", "wb"))

# t-types from catalog

Nair catalog has very specific and inconvenient format. You need a dedicated function for this.

In [168]:
cat = utils.misc.load_Nair('../../bf_data/Nair_and_Abraham_2010/catalog/table2.dat')

inds = cat['ID'].searchsorted(result_arr['id'])
print("Is every element matched?: ", np.all(cat[inds]['ID'] == result_arr['id']))
result_arr['ttype'] = cat[inds]['TT']

# Save the complete catalog
pickle.dump(result_arr, open(out_dir+'result_arr.pcikle', 'wb'))

AttributeError: module 'astrobf.utils' has no attribute 'misc'

# Initialize BO loop -  Measure morphology features

In [ ]:
import torch
import math

# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float

### Initial guesses 


# Run Bayesian Optimization loop